In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
from multiprocessing import Pool
import re
import pickle

In [2]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

my_headers = {
    'authority': 'basketball.realgm.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'referer': 'https://www.google.com/',
    'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8,en-US;q=0.7',
}

In [3]:
url = 'https://basketball.realgm.com/nba/info/salary_cap'

response = session.get(url, headers=my_headers)
print(response)
soup = BeautifulSoup(response.text)

<Response [200]>


# BAE, MLEs and MAXs

In [153]:
max_all = pd.DataFrame(
            pd.read_html(
                str(soup.find_all('table')[13])
                        )[0].iloc[13:30].values)[[0, 1, 2, 3]].rename(
                                                {0: 'season', 1: '0-6 YOS', 2: '7-9 YOS', 3: '10+ YOS'}, axis=1)

In [162]:
mle_all = pd.DataFrame(
            pd.read_html(
                str(soup.find_all('table')[12])
                        )[0].iloc[13:30].values)[[0, 4, 5, 6]].rename(
                                                {0: 'season', 4: 'MLE', 5: 'Tax MLE', 6: 'Room MLE'}, axis=1).ffill(axis ='columns')

In [163]:
bae = pd.DataFrame(
            pd.read_html(
                str(soup.find_all('table')[12])
                        )[0].iloc[13:30].values)[[0, 3]].rename({0: 'season', 3: 'BAE'}, axis=1)

# Minimum salary under different CBAs

In [53]:
url_2005_cba = 'https://basketball.realgm.com/nba/info/minimum_scale/2005'
url_2011_cba = 'https://basketball.realgm.com/nba/info/minimum_scale/2011'
url_2017_cba = 'https://basketball.realgm.com/nba/info/minimum_scale/2017'

In [57]:
response_2005 = session.get(url_2005_cba, headers=my_headers)
print(response_2005)
soup_2005 = BeautifulSoup(response_2005.text)

<Response [200]>


In [65]:
min_2005 = pd.read_html(str(soup_2005.find_all('table')[12]))[0]

In [66]:
response_2011 = session.get(url_2011_cba, headers=my_headers)
print(response_2011)
soup_2011 = BeautifulSoup(response_2011.text)

<Response [200]>


In [67]:
min_2011 = pd.read_html(str(soup_2011.find_all('table')[12]))[0]

In [71]:
min_2011.drop(['2017-2018', '2018-2019', '2019-2020', '2020-2021'], axis=1, inplace=True)

In [72]:
response_2017 = session.get(url_2017_cba, headers=my_headers)
print(response_2017)
soup_2017 = BeautifulSoup(response_2017.text)

<Response [200]>


In [105]:
min_2017 = pd.read_html(str(soup_2017.find_all('table')[12]))[0]

In [106]:
min_2017.rename({'Year 1': '2017-18', 'Year 2': '2018-19',
                 'Year 3': '2019-20', 'Year 4': '2020-21',
                 'Year 5': '2021-22'}, axis=1, inplace=True)

In [107]:
min_2017.loc[0] = min_2017.loc[0].fillna('$815,615')

In [108]:
min_2017.loc[1] = min_2017.loc[1].fillna('$1,378,242')

In [109]:
min_2017.loc[2] = min_2017.loc[2].fillna('$1,618,520')

In [110]:
min_2017.loc[3] = min_2017.loc[3].fillna('$1,752,950')

In [111]:
min_2017['YOS'] = min_2005['YOS']

In [112]:
min_all = min_2005.merge(min_2011, how='left',
                         left_on = 'YOS', right_on = 'YOS').merge(min_2017, how='left',
                                                                  left_on = 'YOS', right_on = 'YOS')

In [174]:
min_all.columns = min_all.columns.str.replace(r'(-20)', "-")

<ipython-input-174-b085fbef51af>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  min_all.columns = min_all.columns.str.replace(r'(-20)', "-")


In [176]:
min_all.rename({'2019-': '2019-20'}, axis=1, inplace=True)

In [190]:
min_all = min_all.T.reset_index().iloc[1:].rename({'index': 'season'}, axis=1)

# Rename rows and columns for final merge

In [194]:
max_all.columns = max_all.columns.str.replace('YOS', 'MAX')

In [220]:
max_all['season'] = max_all['season'].apply(lambda x: x[:5] + x[-2:])

In [222]:
mle_all['season'] = mle_all['season'].apply(lambda x: x[:5] + x[-2:])
bae['season'] = bae['season'].apply(lambda x: x[:5] + x[-2:])

In [202]:
min_all = min_all.add_suffix(' MIN').rename({'season MIN': 'season'}, axis=1)

In [211]:
min_all = min_all[::-1].reset_index(drop=True)

In [251]:
salary_categories = max_all.merge(min_all, how='left',
                                  on='season').merge(mle_all, how='left',
                                                    on='season').merge(bae, how='left',
                                                                      on = 'season')
salary_categories.set_index('season', inplace=True)
salary_categories = salary_categories.applymap(lambda x: int(re.sub(r'[^\d.]', '', x)))

In [253]:
salary_categories_dict = salary_categories.to_dict('index')

In [261]:
for K in salary_categories_dict:
    salary_categories_dict[K] = {v: k for k, v in salary_categories_dict[K].items()}

In [263]:
with open('salary_categories_dict.pickle', 'wb') as handle:
    pickle.dump(salary_categories_dict, handle)